In [ ]:
#Importing packages that are needed
import os
import ee
import geemap
from geemap import *

#Authentication of GEE for use of geemap
#ee.Authenticate()
ee.Initialize()

#Setting center of map
Map = geemap.Map(center=(50, 11), zoom=4)

In [ ]:
#Import countries and filter 
countries = ee.FeatureCollection('users/giswqs/public/countries')
DE = countries.filter(ee.Filter.eq('NAME', 'Germany'))
NED = countries.filter(ee.Filter.eq('NAME', 'Netherlands'))
#Map.addLayer(DE, {}, 'DE')
#Map.addLayer(NED, {}, 'NED')

#POI Lobith
a_lon =  6.101863382807894
a_lat = 51.84911543823371
Lobith = ee.Geometry.Point(a_lon, a_lat)

#Polygon Lobith
Imp_Lobith = 'C:/Users/../Lobith_Poly.shp'
lobith_poly = geemap.shp_to_ee(Imp_Lobith)

In [ ]:
#INPUT
#BASE
BOUNDS = Lobith
POLYGON = Lobith_Poly
POINT = Lobith
START_DATE = '2017-01-01'
END_DATE = '2022-12-31'

#CLOUDMASKING
AOI = POLYGON
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 50

#OUTPUTS
out_name = 'SSC_LOBITH.csv'

In [ ]:
#Function for clipping sampling location 
def clp (image):
    return image.clip(POLYGON).copyProperties(image,['PRODUCT_ID', 'IMAGE_DATE'])

#Function to calculate SSC and returning it to a band-value with obtained formula
#B2 = BLUE ;B3 = GREEN; B4 = RED; B5 = REDEDGE1
def CalculateSSC(image):
    selected = image.select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12')
    expression = '32.143*(b(4)/b(2))**2 + 39.389*(b(4)/b(2)) - 28.804'
    SSC = selected.expression(expression).rename('SSC')
    return selected.addBands(SSC)

In [ ]:
#FUNCTIONS TO COMBINE CLOUDLESS WITH DATASET
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(START_DATE, END_DATE).filterBounds(POLYGON).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 80).map(clp) 

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [ ]:
#Start of s2cloudless cloud masking function for Sentinel

#BUILD COLLECTION ACCORDING TO PARAMETERS 
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [ ]:
#FUNCTION FOR CLOUD PROBABILITY LAYER
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [ ]:
#FUNCTION FOR CLOUD SHADOW COMPONENT
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [ ]:
#FINAL MASK WITH CLOUDS AND CLOUD SHADOWS
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [ ]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [ ]:
#DISPLAY
#s2_sr_cld_col_eval_disp = s2_sr_cld_col.map(add_cld_shdw_mask)

#display_cloud_layers(s2_sr_cld_col_eval_disp)

In [ ]:
#FUNCTION TO APPLY CLOUD MASK IN EACH IMAGE
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
#COLLECTION PROCESS
s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)).map(CalculateSSC)

#End of s2cloudess cloud masking function

In [ ]:
#Deleting any double dates if present
def colltoday (imcoll):
    imlist = imcoll.toList(imcoll.size())

    def dates (image): 
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(dates).distinct()

    def to_date (imdate):
        d = ee.Date(imdate)
        selcoll = imcoll.filterDate(d, d.advance(1, "day"))
        immos = selcoll.mosaic()
        return immos.set("ImgDate", selcoll.first().date().format(),"ListDate", d.format("YYYY-MM-dd"))
    
    mosaic_imcoll = unique_dates.map(to_date) 
        
    return ee.ImageCollection(mosaic_imcoll)

SSCLOBITH_Byday = colltoday(s2_sr)

In [ ]:
#Setting location for export
out_dir = os.path.expanduser('C:/Users/')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_sentinel_stats = os.path.join(out_dir, out_name)

In [ ]:
#xporting the median band values from selected polygons to appointed location
def zonal_img(img): 
    fc = geemap.zonal_statistics(
    img,
    POLYGON,
    out_sentinel_stats,
    statistics_type='MEDIAN',
    scale=10,
    return_fc=True,)
    def setprop(ft): return ft.set("imageDate",img.get("ImgDate"))
    return fc.map(setprop)

out_fc = SSCLOBITH_Byday.map(zonal_img).flatten().filter(ee.Filter.neq('B3', None));   # ('0' for SUM; 'None' for MEDIAN)

geemap.ee_to_csv(out_fc, filename=out_sentinel_stats)